# PDF to Markdown Conversion

This notebook converts downloaded PDFs to markdown format using PyMuPDF. It also performs language detection to filter out incorrectly classified articles, keeping only those in the target language.

## Setup

### Imports

In [1]:
import yaml
import pymupdf
import pymupdf4llm
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from lingua import LanguageDetectorBuilder

Consider using the pymupdf_layout package for a greatly improved page layout analysis.


### Configuration and Paths

In [2]:
# Load configuration
with open("../config.yaml") as f:
    config = yaml.safe_load(f)

# Set up project paths
project_root = Path.cwd().parent
PDFS_DIR = project_root / config["PDFS_DIR"]
EXTRACTED_DIR = project_root / config["EXTRACTED_DIR"]

# Progress bar format
PROGRESS_BAR_FORMAT = "{desc:<25}{percentage:3.0f}%|{bar:20}{r_bar}"

## Helper Functions

In [3]:
def detect_language(text: str, detector):
    """Detect the language of the given text"""
    result = detector.detect_language_of(text)
    return result.iso_code_639_1.name.lower() if result else None

## Process PDFs

Convert PDFs to markdown, filter by language, and display processing statistics.

In [4]:
processing_stats = []
detector = LanguageDetectorBuilder.from_all_languages().build()

for lang_code, lang_config in config["LANGUAGES"].items():
    lang_pdf_dir = PDFS_DIR / lang_code
    lang_extracted_dir = EXTRACTED_DIR / lang_code
    lang_extracted_dir.mkdir(parents=True, exist_ok=True)

    if not lang_pdf_dir.exists():
        continue

    pdf_files = list(lang_pdf_dir.glob("*.pdf"))
    kept_count = 0
    wrong_lang_count = 0
    error_count = 0

    for pdf_path in tqdm(
        pdf_files, 
        total=len(pdf_files), 
        desc=f"Processing {lang_config['name']}", 
        bar_format=PROGRESS_BAR_FORMAT
    ):
        try:
            # Convert PDF to markdown
            with pymupdf.open(str(pdf_path)) as doc:
                md_text = pymupdf4llm.to_markdown(doc)

            # Verify language
            detected_code = detect_language(md_text, detector)
            if detected_code == lang_code:
                # Rename PDF and save markdown
                new_pdf_path = lang_pdf_dir / f"{kept_count}.pdf"
                pdf_path.rename(new_pdf_path)

                md_path = lang_extracted_dir / f"{kept_count}.md"
                md_path.write_text(md_text, encoding="utf-8")
                kept_count += 1
            else:
                # Remove PDF if wrong language
                pdf_path.unlink()
                wrong_lang_count += 1

        except Exception as e:
            print(f"Error converting PDF to markdown: {str(e)}")
            pdf_path.unlink(missing_ok=True)
            error_count += 1

    processing_stats.append(
        {
            "Language": lang_config["name"],
            "Total PDFs": len(pdf_files),
            "Kept": kept_count,
            "Wrong Language": wrong_lang_count,
            "Errors": error_count,
            "Success Rate": (
                f"{kept_count / len(pdf_files) * 100:.1f}%" if pdf_files else "0%"
            ),
        }
    )


Processing Tamil           1%|                    | 3/482 [00:06<16:17,  2.04s/it]

Error converting PDF to markdown: [WinError 183] Cannot create a file when that file already exists: 'c:\\Users\\Admin\\Desktop\\2025-2026_LucasGranucci\\Averroes-AI\\data\\pdfs\\ta\\1.-மூன்று-படிகை்-எழுதுதல்-ஆய்வுக்கட்டுமர-எழுதுை்வழிமுமற-குறித்த-வமரவு.pdf' -> 'c:\\Users\\Admin\\Desktop\\2025-2026_LucasGranucci\\Averroes-AI\\data\\pdfs\\ta\\1.pdf'


Processing Tamil           1%|▏                   | 6/482 [00:14<19:36,  2.47s/it]

Error converting PDF to markdown: [WinError 183] Cannot create a file when that file already exists: 'c:\\Users\\Admin\\Desktop\\2025-2026_LucasGranucci\\Averroes-AI\\data\\pdfs\\ta\\10.46632-ctll-4-1-1.pdf' -> 'c:\\Users\\Admin\\Desktop\\2025-2026_LucasGranucci\\Averroes-AI\\data\\pdfs\\ta\\2.pdf'


Processing Tamil           1%|▎                   | 7/482 [00:14<14:21,  1.81s/it]

...

Error converting PDF to markdown: [WinError 183] Cannot create a file when that file already exists: 'c:\\Users\\Admin\\Desktop\\2025-2026_LucasGranucci\\Averroes-AI\\data\\pdfs\\ta\\62.pdf' -> 'c:\\Users\\Admin\\Desktop\\2025-2026_LucasGranucci\\Averroes-AI\\data\\pdfs\\ta\\7.pdf'


Processing Tamil          52%|██████████▎         | 250/482 [07:13<04:30,  1.17s/it]

Error converting PDF to markdown: [WinError 183] Cannot create a file when that file already exists: 'c:\\Users\\Admin\\Desktop\\2025-2026_LucasGranucci\\Averroes-AI\\data\\pdfs\\ta\\63.pdf' -> 'c:\\Users\\Admin\\Desktop\\2025-2026_LucasGranucci\\Averroes-AI\\data\\pdfs\\ta\\7.pdf'


Processing Tamil          53%|██████████▌         | 255/482 [07:37<06:47,  1.79s/it]


KeyboardInterrupt: 

In [ ]:
display(pd.DataFrame(processing_stats))